In [8]:
import pandas as pd
import numpy as np
import timeit

from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Cargamos el Set de Entrenamiento y el Test Final

In [17]:
train = pd.read_csv('modelo.csv')
test_final = pd.read_csv('test_final.csv')

id_aviso_postulante = test_final['id']

## Realizamos pruebas con el Set de Entrenamiento

Particionamos nuestro set de entrenamiento para testear los resultados de forma local. La idea es realizar varias ejecuciones hasta encontrar los hiper-parámetros que consideramos óptimos, para luego utilizarlos con el test final.

In [27]:
test_s = 0.25
random_s = 0
n_samples = 500000

no = train['se_postulo'] == 0
si = train['se_postulo'] == 1

train_no = train[no].sample(n_samples)
train_si = train[si].sample(n_samples)

train_final = pd.merge(train_no, train_si, how='outer')

x = np.array(train_final[['sexo', 'orden_estudio', 'nivel_laboral', 'tipo_de_trabajo']])
y = np.array(train_final['se_postulo'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_s, random_state=random_s)

In [28]:
tiempo = timeit.timeit()

# Creamos el adaptive boosting
ada_boost = AdaBoostClassifier(n_estimators=50)

# Lo entrenamos con nuestro set de entrenamiento
ada_boost.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = ada_boost.predict(x_test)

# Evaluamos la precisión
precision = accuracy_score(y_test, pred)

print("Precision: ", precision)
print("Tiempo de ejecución: ", tiempo)

Precision:  0.686412
Tiempo de ejecución:  0.018945532817724597


## Predecimos el Test Final 

En base a los resultados obtenidos con el set de entrenamiento, utilizaremos los hiper-parámetros que consideramos óptimos para predecir las postulaciones del test final.

In [29]:
x_test = np.array(test_final[['sexo', 'orden_estudio', 'tipo_de_trabajo', 'nivel_laboral']])

ada_boost = AdaBoostClassifier(n_estimators=50)

# Lo entrenamos con la totalidad del set de datos
ada_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = ada_boost.predict(x_test)

## Submit File

Generamos el Submit File para subir a la competencia de Kaggle

In [30]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submit_adaboost.csv', index=False)